In [2]:
from dotenv import load_dotenv
load_dotenv()

True

https://www.sqlitetutorial.net/sqlite-sample-database/

In [3]:
import os
from langchain_community.utilities import SQLDatabase

# 현재 작업 디렉토리를 사용합니다.
current_dir = os.getcwd()

# chinook.db 파일의 전체 경로를 생성합니다.
db_path = os.path.join(current_dir, "chinook.db")

# SQLite 데이터베이스 URI를 생성합니다.
db_uri = f"sqlite:///{db_path}"

# 데이터베이스 연결을 생성합니다.
db = SQLDatabase.from_uri(db_uri)

# 데이터베이스 방언(dialect)을 출력합니다.
print("Database dialect:", db.dialect)

# 사용 가능한 테이블 이름을 출력합니다.
print("Usable table names:", db.get_usable_table_names())

# employees 테이블에서 모든 데이터를 조회합니다.
result = db.run("SELECT * FROM employees;")
print("Query result:", result)

Database dialect: sqlite
Usable table names: ['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'invoices', 'media_types', 'playlist_track', 'playlists', 'tracks']
Query result: [(1, 'Adams', 'Andrew', 'General Manager', None, '1962-02-18 00:00:00', '2002-08-14 00:00:00', '11120 Jasper Ave NW', 'Edmonton', 'AB', 'Canada', 'T5K 2N1', '+1 (780) 428-9482', '+1 (780) 428-3457', 'andrew@chinookcorp.com'), (2, 'Edwards', 'Nancy', 'Sales Manager', 1, '1958-12-08 00:00:00', '2002-05-01 00:00:00', '825 8 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 2T3', '+1 (403) 262-3443', '+1 (403) 262-3322', 'nancy@chinookcorp.com'), (3, 'Peacock', 'Jane', 'Sales Support Agent', 2, '1973-08-29 00:00:00', '2002-04-01 00:00:00', '1111 6 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 5M5', '+1 (403) 262-3443', '+1 (403) 262-6712', 'jane@chinookcorp.com'), (4, 'Park', 'Margaret', 'Sales Support Agent', 2, '1947-09-19 00:00:00', '2003-05-03 00:00:00', '683 10 Street SW', 'Calgary', 'AB', 'Canada', 

In [4]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question":"처음 10명의 직원의 이름을 표시해줘"})
response

'SELECT "FirstName", "LastName" \nFROM employees \nLIMIT 10;'

In [5]:
db.run(response)

"[('Andrew', 'Adams'), ('Nancy', 'Edwards'), ('Jane', 'Peacock'), ('Margaret', 'Park'), ('Steve', 'Johnson'), ('Michael', 'Mitchell'), ('Robert', 'King'), ('Laura', 'Callahan')]"

In [6]:
response = chain.invoke({"question":"직원은 모두 몇명이야"})
db.run(response)

'[(8,)]'

In [7]:
# 실제 DB에 query를 직접 실행하기 위한 도구를 임포트합니다
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

# DB에 직접 쿼리를 실행할 수 있는 도구를 생성합니다
execute_query = QuerySQLDataBaseTool(db=db)

# LLM을 사용하여 자연어 질문을 SQL 쿼리로 변환하는 체인을 생성합니다
write_query = create_sql_query_chain(llm, db)

# SQL 쿼리 생성과 실행을 연결하는 체인을 생성합니다
chain = write_query | execute_query

# 생성된 체인을 사용하여 자연어 질문을 처리하고 결과를 반환합니다
# 이 경우 "직원은 모두 몇명이야"라는 질문에 대한 답을 구합니다
chain.invoke({"question":"직원은 모두 몇명이야"})

'[(8,)]'

In [8]:
# LangChain agent를 활용해서 SQL query를 실행하고 결과를 출력하는 코드

# SQL 데이터베이스와 상호작용할 수 있는 agent를 생성하는 함수를 임포트합니다
from langchain_community.agent_toolkits import create_sql_agent

# SQL agent를 생성합니다
agent_executor = create_sql_agent(
    llm,                    # 언어 모델 (Large Language Model)
    db=db,                  # 연결할 데이터베이스
    agent_type="openai-tools",  # agent 유형을 OpenAI 도구로 지정
    verbose=False           # 상세한 출력을 비활성화
)

In [9]:
response =  agent_executor.invoke("직원은 모두 몇명이야")["output"]
print(response)

직원은 총 8명입니다.


In [10]:
response =  agent_executor.invoke("고객중에서 가장 구매를 많이한 top 10명과 각각 구매액을 표시해줘")["output"]
print(response)

The top 10 customers who made the most purchases along with their total purchase amounts are:

1. Helena Holý - $49.62
2. Richard Cunningham - $47.62
3. Luis Rojas - $46.62
4. Ladislav Kovács - $45.62
5. Hugh O'Reilly - $45.62
6. Frank Ralston - $43.62
7. Julia Barnett - $43.62
8. Fynn Zimmermann - $43.62
9. Astrid Gruber - $42.62
10. Victor Stevens - $42.62


In [11]:
response =  agent_executor.invoke("가장 많이 팔린 곡 5개는")["output"]
print(response)

가장 많이 팔린 곡 5개는 다음과 같습니다:
1. Wrathchild - 5회 판매
2. The Trooper - 5회 판매
3. The Number Of The Beast - 5회 판매
4. Iron Maiden - 5회 판매
5. Hallowed Be Thy Name - 5회 판매


In [12]:
response =  agent_executor.invoke("가장 많이 팔린 장르는")["output"]
print(response)

The genre that has sold the most is "Rock" with a total of 835 sales.
